In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime as dt
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pd_data_base_path = '/content/drive/MyDrive/Preppin Data/Data/'

In [4]:
balance = pd.read_csv(pd_data_base_path + 'Balance Sheet.csv')

In [5]:
balance.head()
balance.shape

,Unnamed: 0,Account Number,Balance Date,Transaction Value,Balance
0,0,10005367,2023-01-31,NaN,728.25
1,1,10005367,2023-02-02,-1097.60,-369.35
2,2,10005367,2023-02-04,144.75,-224.60
3,3,10005367,2023-02-04,-67.01,-291.61
4,4,10005367,2023-02-05,-108.26,-399.87


(20378, 5)

In [6]:
balance.drop(columns=['Unnamed: 0'], inplace=True)
df = balance.groupby(['Account Number', 'Balance Date']).agg({'Balance':'sum'}).reset_index()
df.head()

,Account Number,Balance Date,Balance
0,10005367,2023-01-31,728.25
1,10005367,2023-02-02,-369.35
2,10005367,2023-02-04,-516.21
3,10005367,2023-02-05,-399.87
4,10005367,2023-02-08,1016.77


In [7]:
df = pd.merge(balance, df, on=['Account Number', 'Balance Date'], how='left')
df.drop(columns=['Balance_x'], inplace=True)
df.rename(columns={'Balance_y':'Balance'}, inplace=True)
df['Balance Date'] = pd.to_datetime(df['Balance Date'])
df.head()

,Account Number,Balance Date,Transaction Value,Balance
0,10005367,2023-01-31,NaN,728.25
1,10005367,2023-02-02,-1097.60,-369.35
2,10005367,2023-02-04,144.75,-516.21
3,10005367,2023-02-04,-67.01,-516.21
4,10005367,2023-02-05,-108.26,-399.87


In [8]:
accounts = df['Account Number'].unique()
balance_dates = pd.date_range('2023-01-31', '2023-02-14')
balance_date = pd.DataFrame({'Account Number':accounts})
balance_date['Balance Date'] = balance_date['Account Number'].apply(lambda x:list(balance_dates))
balance_date = balance_date.explode('Balance Date')

In [9]:
balance_date.head()
balance_date.shape

,Account Number,Balance Date
0,10005367,2023-01-31
0,10005367,2023-02-01
0,10005367,2023-02-02
0,10005367,2023-02-03
0,10005367,2023-02-04


(45000, 2)

In [10]:
df = pd.merge(df, balance_date, on=['Account Number', 'Balance Date'], how='right')
df['Transaction Value'].fillna('null', inplace=True)
df['Balance'].fillna(method='ffill', inplace=True)
df.head()

,Account Number,Balance Date,Transaction Value,Balance
0,10005367,2023-01-31,null,728.25
1,10005367,2023-02-01,null,728.25
2,10005367,2023-02-02,-1097.6,-369.35
3,10005367,2023-02-03,null,-369.35
4,10005367,2023-02-04,144.75,-516.21


In [11]:
user_input = input('Enter the end date (YYYY-MM-DD): ')
try:
    selected_date = dt.strptime(user_input, '%Y-%m-%d')
    print(f'You entered: {selected_date}')
except ValueError:
    print('Invalid date format. Please enter the date in the format YYYY-MM-DD.')
print(f'Output from the selected date i.e. {selected_date}:')
output_df = df[df['Balance Date']==selected_date]
output_df = output_df[['Account Number', 'Balance', 'Transaction Value']]
output_df

Enter the end date (YYYY-MM-DD): 2023-02-01
You entered: 2023-02-01 00:00:00
Output from the selected date i.e. 2023-02-01 00:00:00:


,Account Number,Balance,Transaction Value
1,10005367,728.25,null
18,10011977,676.54,null
35,10024680,414.03,-153.43
51,10031238,576.52,null
68,10034341,390.39,null
...,...,...,...
48081,99734848,354.29,null
48096,99760030,988.00,null
48112,99791709,15.05,null
48128,99877007,478.84,-103.16
